In [1]:
import pandas as pd
from spdynamodb import DynamoTable
from time import sleep
import json
import time
from decimal import Decimal

In [2]:
dt = DynamoTable(profile_name='my-profile')
try:
    dt.select_table('FooBarTable')
    print(dt)
except:
    dt.create_table(
        table_name='FooBarTable',
        partition_key='PK',
        partition_key_type='S',
        sort_key='SK',
        sort_key_type='S',
    )

- Table name: FooBarTable            
- Table arn: arn:aws:dynamodb:us-east-1:953396573731:table/FooBarTable            
- Table creation: 2023-05-10 14:08:54.096000-03:00            
- [{'AttributeName': 'PK', 'KeyType': 'HASH'}, {'AttributeName': 'SK', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'GSI1-PK', 'AttributeType': 'S'}, {'AttributeName': 'GSI1-SK', 'AttributeType': 'S'}, {'AttributeName': 'GSI2-SK', 'AttributeType': 'S'}, {'AttributeName': 'GSI3-SK', 'AttributeType': 'N'}, {'AttributeName': 'PK', 'AttributeType': 'S'}, {'AttributeName': 'SK', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


### Create 3 global secondary indexes

In [ ]:
# 1st Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI1-SK",
    sort_type="S",
    i_name="GSI1"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("1st global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

# 2nd Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI2-SK",
    sort_type="S",
    i_name="GSI2"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("2nd global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

# 3rd Global Secondary Index
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    sort_index="GSI3-SK",
    sort_type="N",
    i_name="GSI3"
)

status = dt.check_status_gsi()
if status == 'CREATING':
    print("3rd global secondary index is being created, this may take a few minutes...")
    start = time.time()
    while status == 'CREATING':
        status = dt.check_status_gsi()
        sleep(30)
end = time.time()
minute = (end - start) / 60
print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

### Users Table

In [ ]:
df_users = pd.read_csv('DesigningDynamoDBTable-1/users.csv')

df_users['PK'] = 'USER#' + df_users['email'].astype(str)
df_users['SK'] = 'USER#' + df_users['email'].astype(str)
df_users.head()

### Orders Table

In [74]:
df_orders = pd.read_csv('DesigningDynamoDBTable-1/orders.csv')

df_orders['PK'] = 'ORDER#' + df_orders['orderId'].astype(str)
df_orders['SK'] = 'ORDER#' + df_orders['orderId'].astype(str)
df_orders['GSI1-PK'] = 'USER#' + df_orders['userId'].astype(str)
df_orders['GSI1-SK'] = 'OSTATUS#' + df_orders['status'].astype(str)
df_orders.head()

,orderId,userId,purchaseDate,status,totalAmount,totalItems,PK,SK,GSI1-PK,GSI1-SK
0,1001,john@example.com,2022-01-01,completed,100.0,2,ORDER#1001,ORDER#1001,USER#john@example.com,OSTATUS#completed
1,1002,jane@example.com,2022-01-02,completed,50.0,1,ORDER#1002,ORDER#1002,USER#jane@example.com,OSTATUS#completed
2,1003,smith@example.com,2022-01-03,shipped,75.0,3,ORDER#1003,ORDER#1003,USER#smith@example.com,OSTATUS#shipped
3,1004,doe@example.com,2022-01-04,pending,20.0,1,ORDER#1004,ORDER#1004,USER#doe@example.com,OSTATUS#pending
4,1005,mike@example.com,2022-01-05,completed,30.0,2,ORDER#1005,ORDER#1005,USER#mike@example.com,OSTATUS#completed


### Order details Table

In [ ]:
df_order_details = pd.read_csv('DesigningDynamoDBTable-1/order_items.csv')

df_order_details['orderId'] = df_order_details['orderId'] + 1000
df_order_details['PK'] = 'ORDER#' + df_order_details['orderId'].astype(str)
df_order_details['SK'] = 'PRODUCT#' + df_order_details['productId'].astype(str)
df_order_details.head()

### Products Table

In [ ]:
df_products = pd.read_csv('DesigningDynamoDBTable-1/products.csv')
values = [100,101,102,200,201,202,301,400,401,402,500,501,502,601,701,702,801,802,900,902]
df_products['productId'] = values
df_products['PK'] = 'PRODUCT#' + df_products['productId'].astype(str)
df_products['SK'] = 'PRODUCT#' + df_products['productId'].astype(str)
df_products['GSI1-PK'] = 'PRODUCT'
df_products['GSI1-SK'] = 'PNAME#' + df_products['name'].astype(str).str.lower()
df_products['GSI2-SK'] =  df_products['category'].astype(str).str.lower()
df_products['GSI3-SK'] =  df_products['price']
df_products.head()

### Add data to DynamoDB

In [ ]:
dt.batch_pandas(dataframe=df_users)

In [75]:
dt.batch_pandas(dataframe=df_orders)

In [ ]:
dt.batch_pandas(dataframe=df_order_details)

In [ ]:
dt.batch_pandas(dataframe=df_products)

### Querying the table

In [4]:
dt.query_items(query="ORDER#1009", to_pandas=True)

,purchaseDate,GSI1PK,totalAmount,orderId,userId,status,SK,GSI1SK,PK,totalItems,quantity,productId
0,2022-01-09,USER#dave@example.com,60,1009,dave@example.com,completed,ORDER#1009,OSTATUS#completed,ORDER#1009,3,NaN,NaN
1,NaN,NaN,27,1009,user9@mail.com,NaN,PRODUCT#901,NaN,ORDER#1009,NaN,2,901
2,NaN,NaN,21,1009,user9@mail.com,NaN,PRODUCT#902,NaN,ORDER#1009,NaN,1,902


In [1]:
from importlib import reload
import spdynamodb
reload(spdynamodb)
reload(spdynamodb._queries)
from spdynamodb import DynamoTable
dt = DynamoTable(profile_name='my-profile')
dt.select_table('FooBarTable')

In [10]:
dt.query(pk_value='PRODUCT', sk_value="category c", consumed_capacity='TOTAL', index_name='GSI2')

GSI1-PK PRODUCT GSI2-SK category c
Consumed Capacity: 0.5


[{'GSI1-SK': 'PNAME#product 7',
  'GSI1-PK': 'PRODUCT',
  'name': 'Product 7',
  'GSI3-SK': 49.99,
  'GSI2-SK': 'category c',
  'category': 'Category C',
  'images': 'image20.jpg,image21.jpg,image22.jpg,image23.jpg',
  'SK': 'PRODUCT#301',
  'amountSold': 90,
  'description': 'Description for Product 7',
  'price': 49.99,
  'PK': 'PRODUCT#301',
  'writersId': 'Writer 7',
  'productId': 301},
 {'GSI1-SK': 'PNAME#product 19',
  'GSI1-PK': 'PRODUCT',
  'name': 'Product 19',
  'GSI3-SK': 4.99,
  'GSI2-SK': 'category c',
  'category': 'Category C',
  'images': 'image66.jpg,image67.jpg,image68.jpg,image69.jpg,image70.jpg',
  'SK': 'PRODUCT#900',
  'amountSold': 150,
  'description': 'Description for Product 19',
  'price': 4.99,
  'PK': 'PRODUCT#900',
  'writersId': 'Writer 9',
  'productId': 900},
 {'GSI1-SK': 'PNAME#product 16',
  'GSI1-PK': 'PRODUCT',
  'name': 'Product 16',
  'GSI3-SK': 39.99,
  'GSI2-SK': 'category c',
  'category': 'Category C',
  'images': 'image54.jpg,image55.jpg,ima